In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf

# x: input, y_: 실제 label 값
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [4]:
# 이미지 x tensor의 차원을 4차원으로 변경.
# 현재는 28 * 28 이미지가 784개의 1차원 배열로 들어오므로,
# 이를 원래의 28 * 28로 바꾸고, 여기에 color channel을 마지막에 추가한 것이라고 생각하면 된다.
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [5]:
# weight, bias를 초기화해서 생성하는 함수들
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [6]:
# conv, pooling을 위한 함수들
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
# 크기가 5x5인 필터를 32개 사용.
W_conv1 = weight_variable([5, 5, 1,32]) # 1은 color, 우리는 흑백이므로 1을 사용.
b_conv1 = bias_variable([32])

In [9]:
# 첫번째 hidden layer.
# activation function으로 relu function 사용.
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [10]:
# 두번째. 이제는 5x5윈도우에 64개 필터 사용.
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [11]:
# 마지막에는 FCNN을 사용. 1024개의 뉴런을 사용하기로 함.
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

In [12]:
# FCNN에 연결하기 위해 h_pool2의 결과를 1차원 배열로 reshape.
# 2차 max pooling까지 끝난 후의 크기가 7*7이며, 64개의 필터를 사용한다.
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [14]:
# overfitting을 방지하기 위한 dropout.
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [15]:
# 최종 softmax
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [17]:
# 정확도 확인
cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()

sess.run(tf.initialize_all_variables())
for i in range(1000):
    batch = mnist.train.next_batch(100)
    
    if i%100 == 0:
        train_accuracy = sess.run(accuracy,
                                  feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print "step: %d, training accuracy: %g" % (i, train_accuracy)
        
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print "test accuracy: %g" % sess.run(accuracy,
                                     feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

step: 0, training accuracy: 0.11
step: 100, training accuracy: 0.92
step: 200, training accuracy: 0.93
step: 300, training accuracy: 0.94
step: 400, training accuracy: 0.91
step: 500, training accuracy: 0.95
step: 600, training accuracy: 0.96
step: 700, training accuracy: 0.97
step: 800, training accuracy: 0.97
step: 900, training accuracy: 0.94
test accuracy: 0.9717
